<a href="https://colab.research.google.com/github/Tessellate-Imaging/monk_v1/blob/master/study_roadmaps/4_image_classification_zoo/Classifier%20-%20American%20Sign%20Language%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents


## Install Monk


## Using pretrained model for classifying american sign language images


## Training a classifier from scratch

# Install Monk

  - git clone https://github.com/Tessellate-Imaging/monk_v1.git

  - cd monk_v1/installation/Linux && pip install -r requirements_cu9.txt
        (Select the requirements file as per OS and CUDA version)

In [ ]:
! git clone https://github.com/Tessellate-Imaging/monk_v1.git

In [ ]:
# If using Colab install using the commands below
! cd monk_v1/installation/Misc && pip install -r requirements_colab.txt

# If using Kaggle uncomment the following command
#! cd monk_v1/installation/Misc && pip install -r requirements_kaggle.txt

# Select the requirements file as per OS and CUDA version when using a local system or cloud
#! cd monk_v1/installation/Linux && pip install -r requirements_cu9.txt

# Used trained classifier for demo

In [ ]:
# Import monk

import os
import sys
sys.path.append("monk_v1/monk/");

In [ ]:
# Download trained weights

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wzxtG8sNb4DyLnSt_QLiJiZ2VIsKnH3W' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1wzxtG8sNb4DyLnSt_QLiJiZ2VIsKnH3W" -O cls_asl_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq cls_asl_trained.zip 

In [ ]:
ls workspace/Project-ASL

In [ ]:
# Pytorch project
from pytorch_prototype import prototype

In [ ]:
# Load project in inference mode

gtf = prototype(verbose=1);
gtf.Prototype("Project-ASL", "Pytorch-resnet18", eval_infer=True);

#Other trained models - uncomment 
#gtf.Prototype("Project-ASL", "Pytorch-resnet34", eval_infer=True);
#gtf.Prototype("Project-ASL", "Pytorch-resnet50", eval_infer=True);

In [ ]:
#Infer

In [ ]:
img_name = "workspace/test/1.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

In [ ]:
img_name = "workspace/test/2.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

In [ ]:
img_name = "workspace/test/3.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

In [ ]:
img_name = "workspace/test/4.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

# Training custom classifier from scratch

# Sign Language Classification with Monk

## Import Monk

In [ ]:
import os
import sys
sys.path.append("./monk_v1/monk")

In [ ]:
from pytorch_prototype import prototype

## Dataset

The ASL alphabet dataset was prepared and hosted by a fellow Kaggler : https://www.kaggle.com/grassknoted/asl-alphabet

In [ ]:
! pip install kaggle

In [ ]:
! kaggle datasets download grassknoted/asl-alphabet

In [ ]:
! unzip -qq asl-alphabet.zip -d asl-alphabet

## Create Experiment

In [ ]:
# Step 1 - Create experiment
ptf = prototype(verbose=1);
ptf.Prototype("asl", "exp1");

In [ ]:
ptf.Default(dataset_path="./asl-alphabet/asl_alphabet_train", model_name="resnet50", freeze_base_network=True, num_epochs=10);

## Train

In [ ]:
ptf.Train()

## Download Workspace

In [ ]:
!zip -r workspace.zip workspace

In [ ]:
from google.colab import files

files.download('workspace.zip')

## Code for Realtime Video Classification

- Place the workspace folder inside your project directory to load the experiment
- Download and setup Monk to your local environment : [DOCS](https://clever-noyce-f9d43f.netlify.com/#/setup/setup)

In [ ]:
import os
import sys
sys.path.append("./monk_v1/monk/");
import psutil
from pytorch_prototype import prototype
from imutils.video import VideoStream
import cv2

ptf = prototype(verbose=1)
ptf.Prototype("asl", "exp1", eval_infer=True);

if __name__ == '__main__':

    vs = VideoStream().start()

    while True:
        # Read Frame and process
        frame = vs.read()
        frame = cv2.resize(frame, (640, 480))

        #Select and store ROI
        cv2.rectangle(frame, (350,50), (600,300), (255,0,0) , 3, 1)

        roi = frame[50:300,350:600]
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        cv2.imwrite("roi.jpg",roi)

        # Run classification
        predictions = ptf.Infer(img_name="roi.jpg", return_raw=False);
        cv2.putText(frame, predictions['predicted_class'],(350,40),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        
        # Display Output
        cv2.imshow('Detection', cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            vs.stop()
            break